In [1]:
from scipy import stats as st
import numpy as np
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt

In [2]:
!pip install --upgrade xlrd
!pip install umap-learn
!pip install kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 2.9 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 2.9 MB/s 
     |████████████████████████████████| 1.1 MB 35.6 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=a59c397acb4c4b5607e7ba8ad8f7d4a89712551bea62f362f6a3edcc47c8ef75
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=d54e13aab0ac218067ee4bd29eb04242c164593006d126d30a5b14f06e41fcc4
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dce

In [3]:
import zipfile
!rm Bin_data.zip
!wget https://github.com/vitvara/tobacco_project_review/raw/main/zip_file/Bin_data.zip
with zipfile.ZipFile("Bin_data.zip","r") as zip_ref:
    zip_ref.extractall()

rm: cannot remove 'Bin_data.zip': No such file or directory
--2022-06-07 16:59:18--  https://github.com/vitvara/tobacco_project_review/raw/main/zip_file/Bin_data.zip
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitvara/tobacco_project_review/main/zip_file/Bin_data.zip [following]
--2022-06-07 16:59:18--  https://raw.githubusercontent.com/vitvara/tobacco_project_review/main/zip_file/Bin_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2118870 (2.0M) [application/zip]
Saving to: ‘Bin_data.zip’

Bin_data.zip        100%[===================>]   2.02M  --.-KB/s    in 0.08s   

2022-06-07 1

In [4]:
category_list = ["Burley", "Turkish", "Virginia"]
category_color = {"Burley":1, "Turkish":2, "Virginia":3}
all = []
c = []
samples = []
for category in category_list:
    for filename in os.listdir(os.path.join("Bin_data", category)):
        df = pd.read_excel(os.path.join("Bin_data", category, filename))
        samples.append(df)
        c.append(category_color[category])
        out = df.iloc[:, 1:].to_numpy().flatten("F").tolist()
        all.append(out)
        if len(out) != 11000:
            print(filename)

In [5]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

def test_raw_data(all, c):
    avgnural = 0
    avgrf = 0
    avglr = 0
    test_case = [0, 12, 34, 42, 75, 100, 150, 123, 343, 1034, 5643]
    highest = []
    lowest = [999999999]
    for i in test_case:
        print("random state:", i)
        X_train, X_test, y_train, y_test = train_test_split(all, c, test_size=0.33, random_state=i, stratify=c)

        reg = LogisticRegression(max_iter=100000000)
        rf = RandomForestClassifier(n_estimators=1000, random_state=0)
        clf = MLPClassifier(alpha=1e-05, hidden_layer_sizes=(20,), random_state=42,max_iter=100000,
                    solver='lbfgs')
        clf.fit(X_train, y_train)
        a = clf.score(X_test, y_test)

        rf.fit(X_train, y_train)
        b = rf.score(X_test, y_test)

        reg.fit(X_train, y_train)
        c1 = reg.score(X_test,y_test)
        print("MLP RF LR scores:", a,b,c1)
        avgnural += a
        avgrf += b
        avglr += c1
        if sum(highest)/3 < sum([a,b,c1])/3:
            highest = [a,b,c1]
        if sum(lowest)/3 > sum([a,b,c1])/3:
            lowest = [a,b,c1]
    print(
        "avg MLP: ", avgnural/len(test_case)
    )
    print(
        "avg RF: ", avgrf/len(test_case)
    )
    print(
        "avg LR: ", avglr/len(test_case)
    )
    print("highest:", highest)
    print("lowest:", lowest)
    return (avgnural/len(test_case) + avgrf/len(test_case) + avglr/len(test_case))/3

In [7]:
def fea_select(data: list, n_feature: int, importance: list) -> list:
    f_i = list(zip(range(len(importance)),importance))
    f_i.sort(key = lambda x : x[1])
    out = []
    for row in data:
        out.append([row[i[0]] for i in f_i[-1:-n_feature:-1]])
    return out

def find_feature_importance(data: list, classes: list, importance: list) -> int:
    max = -9999999
    n_max_feature = None
    for i in [1000, 750, 500, 250, 100, 75, 50, 25, 10, 5]:
        reduced = fea_select(data, i, importance)
        np.array(reduced).shape
        result = test_raw_data(reduced, classes)
        if result > max:
            max = result
            n_max_feature = i
    print("N feature:", n_max_feature)
    target = n_max_feature
    max = -9999999
    n_max_feature = None
    for i in [target+10, target-5, target, target+5, target+10]:
        reduced = fea_select(data, i, importance)
        np.array(reduced).shape
        result = test_raw_data(reduced, classes)
        if result > max:
            max = result
            n_max_feature = i
    print(n_max_feature, "RESULT")
    n_max_feature
    max
    reduced = fea_select(data, n_max_feature, importance)
    np.array(reduced).shape
    result = test_raw_data(reduced, classes)
    return result

In [8]:
rf = RandomForestClassifier(random_state=0)
all_ltuple  = [(i,j) for i,j in zip(all,c) if j in [1,2]]
data = list(np.array(all_ltuple)[:,0])
classes = list(np.array(all_ltuple)[:,1])
rf.fit(data,classes)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  after removing the cwd from sys.path.


RandomForestClassifier(random_state=0)

In [9]:
result = find_feature_importance(data, classes, rf.feature_importances_)

random state: 0
MLP RF LR scores: 0.7333333333333333 0.6 0.8
random state: 12
MLP RF LR scores: 0.7333333333333333 0.5333333333333333 0.5333333333333333
random state: 34
MLP RF LR scores: 0.7333333333333333 0.5333333333333333 0.7333333333333333
random state: 42
MLP RF LR scores: 0.7333333333333333 0.7333333333333333 0.8
random state: 75
MLP RF LR scores: 0.8666666666666667 0.7333333333333333 0.7333333333333333
random state: 100
MLP RF LR scores: 0.7333333333333333 0.6666666666666666 0.7333333333333333
random state: 150
MLP RF LR scores: 0.7333333333333333 0.8666666666666667 0.6666666666666666
random state: 123
MLP RF LR scores: 0.6 0.6 0.7333333333333333
random state: 343
MLP RF LR scores: 0.8 0.6 0.6666666666666666
random state: 1034
MLP RF LR scores: 0.6666666666666666 0.8666666666666667 0.7333333333333333
random state: 5643
MLP RF LR scores: 0.6666666666666666 0.5333333333333333 0.6666666666666666
avg MLP:  0.7272727272727273
avg RF:  0.6606060606060605
avg LR:  0.7090909090909092
h

In [23]:
f_i = list(zip(range(len(rf.feature_importances_)),rf.feature_importances_))
f_i.sort(key = lambda x : x[1], reverse=True)
selected_feature = f_i[:50]

In [ ]:
selected_feature

In [45]:
from scipy.stats import ttest_ind

reduced = [[ row[i] for i,j in selected_feature] for row in data]
a = [sample for idx,sample in enumerate(reduced) if classes[idx] == 1 ]
b = [sample for idx,sample in enumerate(reduced) if classes[idx] == 2 ]
selected_idx = [idx for idx, pvalue in enumerate(ttest_ind(a=a, b=b).pvalue) if pvalue < 0.1]

In [40]:
ttest_ind(a=a, b=b).pvalue

array([0.13585762, 0.10569111, 0.03861878, 0.27230923, 0.11122833,
       0.63485538, 0.06252678, 0.41235978, 0.12369468, 0.09344878,
       0.06092053, 0.30262569, 0.14984626, 0.06613013, 0.20056925,
       0.04809058, 0.03807582, 0.06664372, 0.18707519, 0.4571835 ,
       0.09193886, 0.47000161, 0.07041175, 0.1920363 , 0.65653366,
       0.63007217, 0.18768433, 0.10894077, 0.16506451, 0.8337846 ,
       0.10755884, 0.14733267, 0.2232018 , 0.82530129, 0.70342569,
       0.05562093, 0.61944177, 0.19745779, 0.11385358, 0.11234001,
       0.2599051 , 0.10187413, 0.29277806, 0.35602031, 0.09461389,
       0.17895267, 0.19417983, 0.76091038, 0.15596573, 0.0511738 ])

In [46]:
len(selected_idx)

13

In [47]:
reduced_ttest = [[ row[i] for i in selected_idx] for row in data]

In [48]:
result = test_raw_data(reduced_ttest, classes)
print(result)

random state: 0
MLP RF LR scores: 0.3333333333333333 0.4 0.4666666666666667
random state: 12
MLP RF LR scores: 0.6 0.4666666666666667 0.4666666666666667
random state: 34
MLP RF LR scores: 0.4666666666666667 0.4666666666666667 0.4666666666666667
random state: 42
MLP RF LR scores: 0.4666666666666667 0.6 0.4666666666666667
random state: 75
MLP RF LR scores: 0.4 0.6 0.4666666666666667
random state: 100
MLP RF LR scores: 0.6666666666666666 0.6666666666666666 0.4666666666666667
random state: 150
MLP RF LR scores: 0.6 0.5333333333333333 0.4666666666666667
random state: 123
MLP RF LR scores: 0.5333333333333333 0.5333333333333333 0.4666666666666667
random state: 343
MLP RF LR scores: 0.6 0.5333333333333333 0.4666666666666667
random state: 1034
MLP RF LR scores: 0.5333333333333333 0.5333333333333333 0.4666666666666667
random state: 5643
MLP RF LR scores: 0.6666666666666666 0.5333333333333333 0.4666666666666667
avg MLP:  0.5333333333333333
avg RF:  0.5333333333333333
avg LR:  0.46666666666666673
